In [51]:
import pandas as pd

# Ruta del archivo CSV
csv_file = '/content/ratings.csv'

# Leer el archivo CSV en un DataFrame de pandas, limitando a 1000 filas
df = pd.read_csv(csv_file, nrows=50000)

# Mostrar los primeros 5 registros del dataframe
df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     50000 non-null  int64  
 1   movieId    50000 non-null  int64  
 2   rating     50000 non-null  float64
 3   timestamp  50000 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 1.5 MB


In [53]:
df.count()

userId       50000
movieId      50000
rating       50000
timestamp    50000
dtype: int64

In [54]:
pip install scikit-surprise

In [55]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [56]:
# Crear un objeto Reader para indicar los rangos de calificación posibles
reader = Reader(rating_scale=(1, 5))

In [57]:
# Crear el dataset utilizando el objeto Reader y los datos
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [58]:
# Dividir el dataset en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

In [59]:
# Crear y entrenar el modelo SVD
model = SVD()
model.fit(trainset)


In [60]:
# Hacer predicciones para un usuario específico (por ejemplo, usuario con ID 1)
user_id = 1
anti_testset = trainset.build_anti_testset()
anti_testset = filter(lambda x: x[0] == user_id, anti_testset)
predictions = model.test(anti_testset)


In [61]:
# Obtener las recomendaciones ordenadas por las predicciones de calificación
top_n = 10
recommended_movies = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]

# Mostrar las recomendaciones
print(f"Recomendaciones para el usuario con ID {user_id}:")
for movie in recommended_movies:
    print(f"Película ID: {movie.iid}, Predicción de calificación: {movie.est}")


Recomendaciones para el usuario con ID 1:
Película ID: 318, Predicción de calificación: 5
Película ID: 1201, Predicción de calificación: 4.887564875922392
Película ID: 1213, Predicción de calificación: 4.85252363711775
Película ID: 6016, Predicción de calificación: 4.849817218428812
Película ID: 1228, Predicción de calificación: 4.840848124477694
Película ID: 1193, Predicción de calificación: 4.838021659593236
Película ID: 296, Predicción de calificación: 4.785829613710134
Película ID: 1198, Predicción de calificación: 4.77599272445861
Película ID: 1258, Predicción de calificación: 4.7352197045451945
Película ID: 1136, Predicción de calificación: 4.728452725196563


In [62]:
# Lista de IDs de usuarios para los que deseamos obtener recomendaciones
user_ids = [1, 2, 3, 4]

for user_id in user_ids:
    anti_testset = trainset.build_anti_testset()
    anti_testset = filter(lambda x: x[0] == user_id, anti_testset)
    predictions = model.test(anti_testset)

    top_n = 10
    recommended_movies = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]

    print(f"Recomendaciones para el usuario con ID {user_id}:")
    for movie in recommended_movies:
        print(f"Película ID: {movie.iid}, Predicción de calificación: {movie.est}")
    print("\n")


Recomendaciones para el usuario con ID 1:
Película ID: 318, Predicción de calificación: 5
Película ID: 1201, Predicción de calificación: 4.887564875922392
Película ID: 1213, Predicción de calificación: 4.85252363711775
Película ID: 6016, Predicción de calificación: 4.849817218428812
Película ID: 1228, Predicción de calificación: 4.840848124477694
Película ID: 1193, Predicción de calificación: 4.838021659593236
Película ID: 296, Predicción de calificación: 4.785829613710134
Película ID: 1198, Predicción de calificación: 4.77599272445861
Película ID: 1258, Predicción de calificación: 4.7352197045451945
Película ID: 1136, Predicción de calificación: 4.728452725196563


Recomendaciones para el usuario con ID 2:
Película ID: 1213, Predicción de calificación: 4.419656143192218
Película ID: 1136, Predicción de calificación: 4.409606958062017
Película ID: 1201, Predicción de calificación: 4.355614156547158
Película ID: 750, Predicción de calificación: 4.329198523957037
Película ID: 318, Predic

In [64]:
from surprise.model_selection import cross_validate
import numpy as np


# Realizar validación cruzada con 5 folds
cv_results = cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True)

# Obtener el RMSE promedio del modelo
avg_rmse = np.mean(cv_results['test_rmse'])
print(f"RMSE promedio del modelo: {avg_rmse}")


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8960  0.8807  0.8957  0.8977  0.8819  0.8904  0.0075  
Fit time          0.80    0.78    0.80    1.09    0.86    0.87    0.12    
Test time         0.07    0.08    0.06    0.13    0.06    0.08    0.02    
RMSE promedio del modelo: 0.8904011525208457


In [65]:
from surprise.model_selection import GridSearchCV

# Definir los hiperparámetros a probar
param_grid = {'n_factors': [50, 100, 150], 'n_epochs': [20, 30, 40], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.01, 0.02, 0.05]}

# Crear el objeto GridSearchCV para el modelo SVD
grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE'], cv=5, n_jobs=-1)

# Realizar la búsqueda de cuadrícula en los hiperparámetros
grid_search.fit(data)

# Obtener los mejores hiperparámetros y el mejor RMSE
best_params = grid_search.best_params['rmse']
best_rmse = grid_search.best_score['rmse']

print(f"Mejores hiperparámetros: {best_params}")
print(f"Mejor RMSE: {best_rmse}")


Mejores hiperparámetros: {'n_factors': 100, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.05}
Mejor RMSE: 0.8779691638783452


In [67]:
# Crear un objeto Reader para indicar los rangos de calificación posibles
reader = Reader(rating_scale=(1, 5))

# Crear el dataset utilizando el objeto Reader y los datos
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Dividir el dataset en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

# Crear y entrenar el modelo SVD con los mejores hiperparámetros
model = SVD(n_factors=100, n_epochs=40, lr_all=0.01, reg_all=0.05)
model.fit(trainset)

# Realizar validación cruzada con 5 folds
cv_results = cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True)

# Obtener el RMSE promedio del modelo
avg_rmse = np.mean(cv_results['test_rmse'])
print(f"RMSE promedio del modelo: {avg_rmse}")

# Lista de IDs de usuarios para los que deseamos obtener recomendaciones
user_ids = [2, 3, 4]

for user_id in user_ids:
    anti_testset = trainset.build_anti_testset()
    anti_testset = filter(lambda x: x[0] == user_id, anti_testset)
    predictions = model.test(anti_testset)

    top_n = 10
    recommended_movies = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]

    # Obtener las películas y sus predicciones de calificación
    movie_ids = [movie.iid for movie in recommended_movies]
    ratings = [movie.est for movie in recommended_movies]


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8861  0.8749  0.8691  0.8844  0.8795  0.8788  0.0062  
Fit time          1.80    1.59    1.59    1.60    1.59    1.63    0.08    
Test time         0.07    0.38    0.07    0.07    0.07    0.13    0.12    
RMSE promedio del modelo: 0.8788296058283269
